In [1]:
import geopandas
import pandas as pd
import glob
import shapefile
from tqdm import tqdm
import numpy as np
import pickle
import xlrd3

In [2]:
# read poi from shp files
shp_files = glob.glob('/home/yangwenhao/local/project/weibo_filter/app/data/tianjin/*.shp')

places_names = []
for f in tqdm(shp_files):
    x = geopandas.read_file(f, engine="pyogrio")
    places_names.append(x.name)
    
names = pd.concat(places_names, axis=0)

names.to_csv('tianjin.csv', index=False)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 23/23 [00:38<00:00,  1.66s/it]


In [9]:
names = pd.read_csv('tianjin.csv').name

In [ ]:
greenbelt = 'greenbelt'
greenbelt_list = pd.read_excel('/home/yangwenhao/local/project/weibo_filter/app/data/' + greenbelt + '.xls')

In [10]:
names.head(2)

0    元记电气焊补胎钣金
1       元汇津通卸载
Name: name, dtype: object

In [16]:
location_names = greenbelt_list.name # names

# 'first_2w'
for lst in ['first_2w', 'noimg_6w']:
    data = xlrd3.open_workbook('/home/yangwenhao/local/project/weibo_filter/app/data/' + lst + '.xls')
    table = data.sheets()[0]          #通过索引顺序获取
    
    # get full-text tweets
    texts = []
    test_idx = 5 if lst == 'first_2w' else 6
    for i in range(1, table.nrows):
        texts.append(table.row(i)[test_idx].value)

    texts = pd.DataFrame(texts, columns=['text'])
    # texts.insert(1, 'places', pd.Series([[]]*len(texts)))
    
    # match all palces for all lines
    places = []
    sparse_matirx = []
    for i in tqdm(location_names.unique()): #[297604:]
        try:
            result = texts.text.str.contains(i)
            if result.to_numpy().sum() > 0:
                places.append(i)
                sparse_matirx.append(result.to_numpy())
        except Exception as e:
            continue
    
    # save match results with pickle
    print("出现的地点数: ", len(places), " 记录的列数:", len(sparse_matirx))
    with open('place2index_%s_green.pickle' % lst, 'wb') as f:
        pickle.dump([places, sparse_matirx], f)
        
    # print the most places in all tweets
    name2num = []
    for i,p in enumerate(places):
        name2num.append(sparse_matirx[i].sum())

    name2num = np.array(name2num)
    order_num = np.flip(np.argsort(name2num))
    np_places = np.array(places)
    numofprint = 5
    for p, n in zip(np_places[order_num][:numofprint], name2num[order_num][:numofprint]):
        print(p, ":", n)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 124/124 [00:00<00:00, 801.17it/s]


22 22
天津动物园 : 15
天津水上公园 : 15
滨海文化中心 : 11
水西公园 : 10
西沽公园 : 9


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 124/124 [00:00<00:00, 855.72it/s]

22 22
天津动物园 : 12
滨海文化中心 : 9
人民公园 : 7
天津文化中心 : 5
天津水上公园 : 4
